# Nucleus-corrected *Afρ* values

Given a measurement of a comet with an estimated nuclear size, what is the *Afρ* quantity with and without the nucleus brightness removed?

## Background

Given a circular aperture of radius *ρ* centered on the comet, *Afρ* is the product of albedo, filling factor, and aperture radius in units of length projected to the distance of the comet.  With some caveats, it is useful for inter-comparing cometary photometry (A'Hearn et al. 1984; Fink and Rubin 2012).

In some cases, a cometary nucleus may contribute a significant fraction of the observed brightness.  Such a scenario may bias one's interpretation of the *Afρ* value.  If the nucleus size or brightness is known, it ca be removed before computing *Afρ*.  `sbpy` can help address this problem.

## The observational data and nominal *Afρ*

Given the following observational circumstances, what is the *Afρ* quantity, neglecting any potential nuclear contribution?

* *V* = 18 mag (VEGA magnitude system)
* *ρ* = 5 arcsec
* heliocentric distance, $r_h$ = 2.5 au
* observer-target distance, *Δ* = 3.5 au
* phase angle, 3.5°

In addition, use the Schleicher-Marcus (née, Halley-Marcus) phase function to correct the photometry to a phase angle of 0°.

In [1]:
import astropy.units as u
from sbpy.activity import Afrho, phase_HalleyMarcus
from sbpy.data import Ephem
import sbpy.units as sbu

m_comet = 18 * sbu.VEGAmag
bandpass = "Johnson V"
eph = Ephem.from_dict(dict(rh=2.5 * u.au, delta=3.5 * u.au, phase=3.5 * u.deg))
rho = 5 * u.arcsec

afrho = Afrho.from_fluxd(bandpass, m_comet, rho, eph, phasecor=True, Phi=phase_HalleyMarcus)

print(f'Afρ based on nominal photometry: {afrho:.1f}')

Afρ based on nominal photometry: 77.5 cm


To be sure we understand what went into this calculation behind the scenes, let's retrieve the apparent magnitude of the Sun from `sbpy`'s calibration system.

In [2]:
from sbpy.calib import Sun

sun = Sun.from_default()
_, _, m_sun = sun.observe_filter_name(bandpass, unit=sbu.VEGAmag)

print(f"Apparent magnitude of the Sun: {m_sun:.2f}")

Apparent magnitude of the Sun: -26.76 mag(VEGA)


## Nucleus corrected photometry

Let's assume on a previous observing run we measured the nucleus size when the comet was inactive.  With the IAU HG model, we estimated an absolute magnitude *H*=14 mag (again, VEGA magnitude system), for a phase function parameter *G*=0.15.

Remove the nucleus contribution from our photometry and re-compute *Afρ*.

In [3]:
import numpy as np
from sbpy.photometry import HG

model = HG(H=14, G=0.15)
m_nucleus = 5 * np.log10(eph['rh'].value * eph['delta'].value)[0] + model(eph['phase'][0])
m_nucleus = m_nucleus * sbu.VEGAmag

# using `.physical` to convert magnitudes into linear units
coma = m_comet.physical - m_nucleus.physical

# convert back to VEGA magnitudes
m_coma = coma.to(sbu.VEGAmag)

# recalculate Afrho
afrho_coma = Afrho.from_fluxd(bandpass, m_coma, rho, eph, phasecor=True, Phi=phase_HalleyMarcus)

print(f"""Total comet magnitude: {m_comet:.2f}
Apparent magnitude of nucleus: {m_nucleus:.2f}
Apparent magnitude of the coma: {m_coma:.2f}

Nucleus-corrected *Afρ*: {afrho_coma:.1f}

""")

Total comet magnitude: 18.00 mag(VEGA)
Apparent magnitude of nucleus: 19.05 mag(VEGA)
Apparent magnitude of the coma: 18.52 mag(VEGA)

Nucleus-corrected *Afρ*: 48.0 cm


